<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HANDWRITTEN DIGITS RECOGNITION USING CNN
---

Handwritten digits classification using **CNN**. This solution is implemented in `Julia` using the `Flux.jl` library along `CUDA.jl` for GPU computing support.

In [ ]:
versioninfo()

In [ ]:
# ;wget -O Project.toml https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml

In [ ]:
# ;sed -i -e '/^Pl[uo]/d' -e '/^Image/d' Project.toml

In [ ]:
pkgs = """ \
[deps]
BSON = "fbb218c0-5317-5bc6-957e-2ee96dd4b1f0"
CUDA = "052768ef-5323-5732-b1bb-66c8b64840ba"
Flux = "587475ba-b771-5e3f-ad9e-33799f191a9c"
ImageInTerminal = "d8c32880-2388-543b-8c61-d9f865259254"
JLD2 = "033835bb-8acc-5ee8-8aae-3f567f8a3819"
MLDatasets = "eb30cadb-4394-5ae3-aed4-317e484a6458"
Markdown = "d6f4376e-aef5-505a-96c1-9c027394607a"
ProgressMeter = "92933f4c-e287-5a05-a399-4b506db050ca"
Statistics = "10745b16-79ce-11e8-11f9-7d13ad32a3b2"
WGLMakie = "276b4fcb-3e11-5398-bf8b-a0c2d153d008"
cuDNN = "02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"
"""

open("Project.toml", "w") do file
    write(file, pkgs)
end

In [ ]:
import Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.update(); Pkg.precompile(); Pkg.resolve(); Pkg.status()

In [ ]:
Pkg.precompile(); Pkg.resolve()

Import the machine learning library `Flux`

In [ ]:
using Flux
using Flux: DataLoader
using Flux: onecold, onehotbatch

In [ ]:
using CUDA
CUDA.versioninfo()

In [ ]:
Base.@kwdef mutable struct HyperParams
    η = 3f-3                # Learning rate
    batchsize = 64          # Batch size
    epochs = 8              # Number of epochs
    split = :train          # Split data into `train` and `test`
end

Load the **MNIST** dataset

In [ ]:
using MLDatasets

In [ ]:
d = MNIST()

In [ ]:
function get_data(; kws...)
    args = HyperParams(; kws...);
    # Split and normalize data
    data = MNIST(split=args.split);
    X, y = data.features ./ 255, data.targets;
    X = reshape(X, (28, 28, 1, :));
    y = onehotbatch(y, 0:9);
    loader = DataLoader((X, y); batchsize=args.batchsize, shuffle=true) |> gpu;
    return loader
end

In [ ]:
train_loader = get_data();
test_loader = get_data(split=:test);

In [ ]:
x1, y1 = first(get_data());
# CPU => Array...
# GPU => CuArray...
typeof((x1, y1))

Transform sample training data to an image. View the image and check the corresponding digit value.

In [ ]:
using Statistics

In [ ]:
idx = rand(1:6_000, 3)

In [ ]:
using ImageInTerminal

In [ ]:
convert2image(d, idx)

In [ ]:
"Digit are $(d.targets[idx])"

**CNN** ARCHITECTURE

The input `X` is a batch of images with dimensions `(width=28, height=28, channels=1, batchsize)`

In [ ]:
fc = prod(Int.(floor.([28/4 - 2, 28/4 - 2, 16]))) # 2^{\# max-pool}

In [ ]:
model = Chain(
            Conv((5, 5), 1 => 16, relu),  # (28-5+1)x(28-5+1)x16 = 24x24x16
            MaxPool((2, 2)),              # 12x12x16
            Conv((3, 3), 16 => 16, relu), # (12-3+1)x(12-3+1)x16 = 10x10x16
            MaxPool((2, 2)),              # 5x5x16
            Flux.flatten,                 # 400
            Dense(fc => 64, relu),
            Dense(64 => 32, relu),
            Dense(32 => 10)
) |> gpu

In [ ]:
using ProgressMeter: Progress, next!

In [ ]:
function train(; kws...)
    args = HyperParams(; kws...)
    # Define the loss function
    l(α, β) = Flux.logitcrossentropy(α, β)
    # Define the accuracy metric
    acc(α, β) = mean(onecold(α) .== onecold(β))
    # Optimizer
    optim_state = Flux.setup(Adam(args.η), model);

    vec_loss = []
    vec_acc = []

    for epoch in 1:args.epochs
        printstyled("\t***\t === EPOCH $(epoch) === \t*** \n", color=:magenta, bold=true)
        @info "TRAINING"
        prg_train = Progress(length(train_loader))
        for (X, y) in train_loader
            loss, grads = Flux.withgradient(model) do m
                ŷ = m(X);
                l(ŷ, y);
            end
            Flux.update!(optim_state, model, grads[1]); # Upd `W` and `b`
            # Show progress meter
            next!(prg_train, showvalues=[(:loss, loss)])
        end
        @info "TESTING"
        prg_test = Progress(length(test_loader))
        for (X, y) in test_loader
            ŷ = model(X);
            push!(vec_loss, l(ŷ, y));  # log `loss` value -> `vec_loss` vector
            push!(vec_acc, acc(ŷ, y)); # log `accuracy` value -> `vec_acc` vector
          	# Show progress meter
            next!(prg_test, showvalues=[(:loss, vec_loss[end]), (:accuracy, vec_acc[end])])
        end
    end
    return vec_loss, vec_acc
end

In [ ]:
vec_loss, vec_acc = train()

Plot results

In [ ]:
# Load the package
using WGLMakie

# Create the plot
fig = Figure(size = (800, 400))
ax1 = Axis(
    fig[1, 1],
    xlabel = "Iteration",
    title = "Test Loss vs. Iteration"
)
ax2 = Axis(
    fig[2, 1],
    xlabel = "Iteration",
    title = "Test Accuracy vs. Iteration"
)

# Plot the `vec_loss`
lines!(ax1, vec_loss, color = :red, linewidth = 2)

# Plot the `acc_loss`
lines!(ax2, acc_loss, color = :blue, linewidth = 2)

# Display the figure
fig

In [ ]:
#=
idx = rand(1:1000, 16)
xs = test_loader.DataLoader[idx][1]
yp = xs |> model |> softmax |> out -> onecold(out, 0:9) |> cpu
ys = onecold(test_loader.data[idx][2]) .- 1 |> cpu;
=#

In [ ]:
#=
for i ∈ eachindex(yp)
    @info "**Prediction is $(yp[i]). Label is $(ys[i]).**"
end
=#

Save and load the model

In [ ]:
folder = "models"
isdir(folder) || mkdir(folder)

In [ ]:
using JLD2

filename_1 = joinpath(folder, "cnn.jld2")
JLD2.jldsave(filename_1; cnn_state = Flux.state(model) |> cpu)
cnn_state = JLD2.load(filename_1, "cnn_state")

In [ ]:
using BSON: @save, @load

filename_2 = joinpath(folder, "cnn.bson")
@save filename_2 model
@load filename_2 model